In [0]:
%pip install -U -q databricks-langchain langchain==0.3.7 faiss-cpu wikipedia langgraph==0.5.3  databricks_langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.18.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

### Tool Use

In [0]:
import os
import sys
import asyncio
import nest_asyncio
from typing import Optional, List

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor


from databricks_langchain import ChatDatabricks 


In [0]:
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-1-8b-instruct" # Model Serving endpoint name; other option see "Serving" under AI/ML tab (e.g. databricks-gpt-oss-20b)

llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME, temperature=0.2)

In [0]:
# --- Define a Tool ---
@langchain_tool
def search_information(query: str) -> str:
    """
    Provides factual information on a given topic. Use this tool to find answers to phrases
    like 'capital of France' or 'weather in London?'.
    """
    print(f"\n--- 🛠️ Tool Called: search_information with query: '{query}' ---")
    simulated_results = {
        "weather in london": "The weather in London is currently cloudy with a temperature of 15°C.",
        "capital of france": "The capital of France is Paris.",
        "population of earth": "The estimated population of Earth is around 8 billion people.",
        "tallest mountain": "Mount Everest is the tallest mountain above sea level.",
        "default": f"Simulated search result for '{query}': No specific information found, but the topic seems interesting.",
    }
    result = simulated_results.get(query.lower(), simulated_results["default"])
    print(f"--- TOOL RESULT: {result} ---")
    return result

In [0]:
tools: List = [search_information]

In [0]:
# --- Create a Tool-Calling Agent ---
if not llm:
    raise RuntimeError("LLM not initialized; cannot create agent.")

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools when useful."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, agent_prompt)

# NOTE: With Databricks, tool-calling support depends on the serving model/endpoint.
# If your endpoint doesn't support tool calls, the agent may fail or ignore tools.
agent_executor = AgentExecutor(agent=agent, verbose=True, tools=tools)

In [0]:
# --- Run the agent concurrently ---
async def run_agent_with_tool(query: str):
    print(f"\n--- 🏃 Running Agent with Query: '{query}' ---")
    try:
        response = await agent_executor.ainvoke({"input": query})
        print("\n--- ✅ Final Agent Response ---")
        print(response["output"])
    except Exception as e:
        print(f"\n🛑 An error occurred during agent execution: {e}")

In [0]:
tasks = [
    run_agent_with_tool("What is the capital of France?"),
    run_agent_with_tool("What's the weather like in London?"),
    run_agent_with_tool("Tell me something about dogs."),
]
await asyncio.gather(*tasks)


--- 🏃 Running Agent with Query: 'What is the capital of France?' ---

--- 🏃 Running Agent with Query: 'What's the weather like in London?' ---

--- 🏃 Running Agent with Query: 'Tell me something about dogs.' ---


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...

Invoking: `search_information` with `{'query': 'weather in London'}`



--- 🛠️ Tool Called: search_information with query: 'weather in London' ---
--- TOOL RESULT: The weather in London is currently cloudy with a temperature of 15°C. ---
The weather in London is currently cloudy with a temperature of 15°C.
Invoking: `search_information` with `{'query': 'capital of France'}`



--- 🛠️ Tool Called: search_information with query: 'capital of France' ---
--- TOOL RESULT: The capital of France is Paris. ---
The capital of France is Paris.Dogs are a popular pet and are known for their loyalty and affection towards their owners. They come in a wide range of breeds,

[None, None, None]

### Planning

In [0]:
import os
import getpass
from openai import OpenAI

# --- Auth (recommended: env var, not hardcoded) ---
OPENAI_API_KEY = "sk-YOUR-OPENAI-API-KEY"

client = OpenAI(api_key=OPENAI_API_KEY)

In [0]:
# --- Define the agent's role and the user's research question ---
system_message = """You are a professional researcher preparing a structured, data-driven report.
Focus on data-rich insights, use reliable sources, and include inline citations."""
user_query = "Research the economic impact of semaglutide on global healthcare systems."


In [0]:
# --- Create the Deep Research API call ---
response = client.responses.create(
    model="o3-deep-research-2025-06-26",
    input=[
        {
            "role": "developer",
            "content": [{"type": "input_text", "text": system_message}],
        },
        {
            "role": "user",
            "content": [{"type": "input_text", "text": user_query}],
        },
    ],
    reasoning={"summary": "auto"},
    tools=[{"type": "web_search_preview"}],
)

# --- Access and print the final report ---
final_report = response.output[-1].content[0].text
print(final_report)


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
print("\n--- CITATIONS ---")
annotations = getattr(response.output[-1].content[0], "annotations", None)

if not annotations:
    print("No annotations found in the report.")
else:
    for i, citation in enumerate(annotations, start=1):
        cited_text = final_report[citation.start_index : citation.end_index]
        print(f"\nCitation {i}:")
        print(f"  Cited Text: {cited_text}")
        print(f"  Title: {citation.title}")
        print(f"  URL: {citation.url}")
        print(f"  Location: chars {citation.start_index}–{citation.end_index}")

print("\n" + "=" * 50 + "\n")


In [0]:
# --- INSPECT INTERMEDIATE STEPS ---
print("--- INTERMEDIATE STEPS ---")

# 1) Reasoning step summary (if present)
reasoning_steps = [item for item in response.output if item.type == "reasoning"]
if reasoning_steps:
    print("\n[Found Reasoning Step(s)]")
    for step in reasoning_steps:
        for part in getattr(step, "summary", []) or []:
            print(f"  - {part.text}")
else:
    print("\nNo reasoning steps found.")

In [0]:

# 2) Web search calls (if present)
search_steps = [item for item in response.output if item.type == "web_search_call"]
if search_steps:
    print("\n[Found Web Search Call(s)]")
    for step in search_steps:
        # action/query shape can vary; be defensive
        action = getattr(step, "action", {}) or {}
        query = action.get("query", None)
        print(f"  Query Executed: {query!r}")
        print(f"  Status: {getattr(step, 'status', None)}")
else:
    print("\nNo web search steps found.")

In [0]:
# 3) Code interpreter calls (if present)
code_steps = [item for item in response.output if item.type == "code_interpreter_call"]
if code_steps:
    print("\n[Found Code Execution Step(s)]")
    for step in code_steps:
        print("  Code Input:")
        print(f"  ```python\n{getattr(step, 'input', '')}\n  ```")
        print("  Code Output:")
        print(f"  {getattr(step, 'output', '')}")
else:
    print("\nNo code execution steps found.")